***GENERATED CODE FOR 2ndapriltest PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***OPERATION FUNCTIONS***

In [ ]:
import pyspark
from dask.dataframe import from_pandas
import json


def appendValueToColumn(df, functionsData, applyOn):
    for obj in functionsData:
        df[obj['columnName']] = df[obj['columnName']].astype(
            str) + obj['appendValue']
    return df


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '00C8A2A4-6870-424B-94C2-F77590F9D662', 'max': 'FF814068-2820-4DC9-91BD-DB58F474BF15', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChargeDetailID'}, {'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ChargeDetailID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfEntry', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'DateOfEntry'}, {'feature_label': 'DateOfEntry', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('DateOfEntry')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateOfService', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfService', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'DateOfService'}, {'feature_label': 'DateOfService', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('DateOfService')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'}], 'feature': 'ProviderID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '03151B5D-685B-4211-B01F-7BBB1C4A8A49', 'max': 'CBBD4925-1757-45BC-A853-1C2CA203B9E3', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ProviderID'}, {'feature_label': 'ProviderID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ProviderID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '90932.19', 'stddev': '6557.08', 'min': '67028', 'max': 'J9035', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'B58.01', 'max': 'T66.XXXD', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'B20', 'max': 'Z96.1', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis2')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Cardiology', 'max': 'Pulmonary', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Ophtamology', 'max': 'Ophtamology', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SpecialityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PayerId', 'transformation_label': 'String Indexer'}], 'feature': 'PayerId', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '03E92275-9F7C-4D34-8701-CE4C72CDE9F7', 'max': 'F71F64B6-E8D9-4A9C-A7E3-A518F957DE52', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PayerId'}, {'feature_label': 'PayerId', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PayerId')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'GG', 'transformation_label': 'String Indexer'}], 'feature': 'GG', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'FS', 'max': 'MS', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'GG'}, {'feature_label': 'GG', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('GG')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Sep', 'transformation_label': 'String Indexer'}], 'feature': 'Sep', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'OphtamologyA', 'max': 'OphtamologyA', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Sep'}, {'feature_label': 'Sep', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Sep')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pyspark


def linear_regression(sparkDF, labels, features):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    lr_model = LinearRegression()
    X_train, X_test, y_train, y_test = train_test_split(
        df[features], df[labels])
    lr_model.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % lr_model.score(X_test, y_test))

    data = {'model': lr_model,
            'X_test': X_test,
            'y_test': y_test,
            'label': labels,
            'columnNames': df.columns}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run 2ndapriltestHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'Demodb2020', 'optionalDB': 'Demodb2020', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT sub2.ChargeDetailID,sub2.Age,\r\n\t\t\t\tsub2.Gender, sub2.DateOfEntry, sub2.DateOfService,\r\n\t\t\t\tsub2.ProviderID, sub2.CPTCode,\r\n                CASE WHEN sub2.pdiagnosis1 IS NOT NULL THEN sub2.pdiagnosis1 ELSE 'XXXX' END as pdiagnosis1,\r\n                CASE WHEN sub2.pdiagnosis2 IS NOT NULL THEN sub2.pdiagnosis2 ELSE 'XXXX' END as pdiagnosis2,\r\n\t\t\t\tsub2.DepartmentCategory, sub2.SpecialityCategory,\r\n\t\t\t\tDATEDIFF(day, MIN(sub2.Chargedate), MIN(sub1.PaymentDate)) as days_to_first_post,\r\n\t\t\t\tSub2.PayerId\r\nFROM \r\n(\r\n\t\tSELECT \r\n\t\tDISTINCT cd.ChargeDetailID, Transactiondate as Chargedate,\r\n\t\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age,\r\n\t\tp.Gender, cd.DateOfEntry, cd.DateOfService,\r\n\t\tpp.ProviderID,\r\n\t\tcptc.CPTCode,\r\n\t\tdc.DiagnosisCode as pdiagnosis1,\r\n\t\tdc2.DiagnosisCode as pdiagnosis2,\r\n\t\td.DepartmentCategory,\r\n\t\ts.SpecialityCategory,\r\n\t\tt.PayerId,t.SecondaryPayerID,t.TertiaryPayerID\r\n\t\tfrom Transactions t\r\n\t\tInner join chargedetail cd on t.Chargedetailid = cd.chargedetailid\r\n\t\tleft join transactiontype tt on t.TransactionTypeID=tt.TransactionTypeID\r\n\t\tinner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\t\tinner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\t\tinner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\t\tinner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\t\tinner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\t\tinner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\t\tinner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\t\tWHERE TransactiontypeCategory = 'Charges'\r\n\t\tand t.DateOfEntry between '1/1/2020' and '10/30/2020'  \r\n) as sub2\r\nINNER JOIN\r\n(\r\n\t\tSELECT\r\n\t\t\t t.ChargeDetailID as cid, TransactionDate as PaymentDate,\r\n\t\t\t t.DateOfEntry,\r\n\t\t\t count(*) as total_transactions\r\n\t\tfrom Transactions t\r\n\t\tinner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\t\tWHERE t.Amount > 0.1 \r\n\t\tand tt.TransactionTypeCategory = 'Payments' and transactiondate is not null \r\n\t\tgroup by t.DateOfEntry, t.ChargeDetailID,TransactionDate\r\n) as sub1\r\n on sub1.cid = sub2.ChargeDetailID\r\n where sub2.Gender is not NULL\r\n  Group by sub2.ChargeDetailID,  ChargeDate,sub2.Age,\r\n\t\t\t\tsub2.Gender, sub2.DateOfEntry, sub2.DateOfService,\r\n\t\t\t\tsub2.ProviderID, sub2.CPTCode,sub2.pdiagnosis1, sub2.pdiagnosis2,\r\n\t\t\t\tsub2.DepartmentCategory, sub2.SpecialityCategory,sub1.PaymentDate,\r\n\t\t\t\tSub2.PayerId", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run 2ndapriltestHooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(sqlserver,spark,json.dumps( {"url": null, "source_attributes": {"host": "wsc-sqlserver1.database.windows.net", "port": "1433", "password": "G5gTW5rnKVeAZWXm", "database": "Demodb2020", "optionalDB": "Demodb2020", "user": "Numtra", "qtype": "query", "query": "SELECT DISTINCT sub2.ChargeDetailID,sub2.Age,\r\n\t\t\t\tsub2.Gender, sub2.DateOfEntry, sub2.DateOfService,\r\n\t\t\t\tsub2.ProviderID, sub2.CPTCode,\r\n                CASE WHEN sub2.pdiagnosis1 IS NOT NULL THEN sub2.pdiagnosis1 ELSE 'XXXX' END as pdiagnosis1,\r\n                CASE WHEN sub2.pdiagnosis2 IS NOT NULL THEN sub2.pdiagnosis2 ELSE 'XXXX' END as pdiagnosis2,\r\n\t\t\t\tsub2.DepartmentCategory, sub2.SpecialityCategory,\r\n\t\t\t\tDATEDIFF(day, MIN(sub2.Chargedate), MIN(sub1.PaymentDate)) as days_to_first_post,\r\n\t\t\t\tSub2.PayerId\r\nFROM \r\n(\r\n\t\tSELECT \r\n\t\tDISTINCT cd.ChargeDetailID, Transactiondate as Chargedate,\r\n\t\tFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age,\r\n\t\tp.Gender, cd.DateOfEntry, cd.DateOfService,\r\n\t\tpp.ProviderID,\r\n\t\tcptc.CPTCode,\r\n\t\tdc.DiagnosisCode as pdiagnosis1,\r\n\t\tdc2.DiagnosisCode as pdiagnosis2,\r\n\t\td.DepartmentCategory,\r\n\t\ts.SpecialityCategory,\r\n\t\tt.PayerId,t.SecondaryPayerID,t.TertiaryPayerID\r\n\t\tfrom Transactions t\r\n\t\tInner join chargedetail cd on t.Chargedetailid = cd.chargedetailid\r\n\t\tleft join transactiontype tt on t.TransactionTypeID=tt.TransactionTypeID\r\n\t\tinner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\n\t\tinner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\n\t\tinner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\n\t\tinner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\n\t\tinner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\n\t\tinner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\n\t\tinner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\n\t\tWHERE TransactiontypeCategory = 'Charges'\r\n\t\tand t.DateOfEntry between '1/1/2020' and '10/30/2020'  \r\n) as sub2\r\nINNER JOIN\r\n(\r\n\t\tSELECT\r\n\t\t\t t.ChargeDetailID as cid, TransactionDate as PaymentDate,\r\n\t\t\t t.DateOfEntry,\r\n\t\t\t count(*) as total_transactions\r\n\t\tfrom Transactions t\r\n\t\tinner join Transactiontype tt on t.transactiontypeid = tt.TransactionTypeID\r\n\t\tWHERE t.Amount > 0.1 \r\n\t\tand tt.TransactionTypeCategory = 'Payments' and transactiondate is not null \r\n\t\tgroup by t.DateOfEntry, t.ChargeDetailID,TransactionDate\r\n) as sub1\r\n on sub1.cid = sub2.ChargeDetailID\r\n where sub2.Gender is not NULL\r\n  Group by sub2.ChargeDetailID,  ChargeDate,sub2.Age,\r\n\t\t\t\tsub2.Gender, sub2.DateOfEntry, sub2.DateOfService,\r\n\t\t\t\tsub2.ProviderID, sub2.CPTCode,sub2.pdiagnosis1, sub2.pdiagnosis2,\r\n\t\t\t\tsub2.DepartmentCategory, sub2.SpecialityCategory,sub1.PaymentDate,\r\n\t\t\t\tSub2.PayerId", "dbtype": "mssql", "is_header": "Use Header Line", "server_url": "/numtraPlatform/NumtraPlatformV2/uploads/platform/"}, "DataPrepFile": null, "data_source": "mssql", "startListenerOnly": 1, "FilePath": "/FileStore/platform/extra/606d49704c7413647024a65b1617783131/0part.csv", "requestRatio": 296.0, "totalRows": 147934, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 15, "numberOfRows": 147934, "duplicateRowCount": 0, "stats": [{"column": "ChargeDetailID", "alias": "ChargeDetailID", "type": "String", "max": "FFFF91CB-4D2B-48E7-AAE0-D2CB35E5B8E3", "min": "0000F12A-424D-43D9-A046-BA4208D30CBD", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Age", "alias": "Age", "type": "real", "max": 108.0, "min": 2.0, "mean": 71.48899509240607, "missing": 0.0, "stddev": 14.96}, {"column": "Gender", "alias": "Gender", "type": "String", "max": "M", "min": "F", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "DateOfEntry", "alias": "DateOfEntry", "type": "String", "max": "2020-10-30T00:00:00.000Z", "min": "2020-01-02T00:00:00.000Z", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "DateOfService", "alias": "DateOfService", "type": "String", "max": "2020-10-30T00:00:00.000Z", "min": "2017-08-10T00:00:00.000Z", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "ProviderID", "alias": "ProviderID", "type": "String", "max": "CBBD4925-1757-45BC-A853-1C2CA203B9E3", "min": "03151B5D-685B-4211-B01F-7BBB1C4A8A49", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "CPTCode", "alias": "CPTCode", "type": "String", "max": "S9981", "min": "65222", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "pdiagnosis1", "alias": "pdiagnosis1", "type": "String", "max": "Z98.890", "min": "A51.43", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "pdiagnosis2", "alias": "pdiagnosis2", "type": "String", "max": "Z98.890", "min": "A15.0", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "DepartmentCategory", "alias": "DepartmentCategory", "type": "String", "max": "Pulmonary", "min": "Cardiology", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "SpecialityCategory", "alias": "SpecialityCategory", "type": "String", "max": "Ophtamology", "min": "Ophtamology", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "days_to_first_post", "alias": "days_to_first_post", "type": "real", "max": 355.0, "min": -667.0, "mean": 27.74492679167737, "missing": 0.0, "stddev": 73.35}, {"column": "PayerId", "alias": "PayerId", "type": "String", "max": "FF58D17E-F6AC-4390-999A-AE5378E83883", "min": "012FD8A9-37BE-4D13-B808-7E6742223444", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "GG", "alias": "GG", "type": "String", "max": "MS", "min": "FS", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "Sep", "alias": "Sep", "type": "String", "max": "OphtamologyA", "min": "OphtamologyA", "mean": "", "missing": 0.0, "stddev": ""}]}, "HasBasicStats": 1, "functionsApplied": [{"functionName": "appendvalue", "applyOn": [{"columnName": "Gender", "type": "String", "min": "F", "max": "M", "mean": "-"}], "functionsData": [{"columnName": "Gender", "appendValue": "S", "newColumnName": "GG", "Column": {"columnName": "Gender", "type": "String", "min": "F", "max": "M", "mean": "-"}}]}, {"functionName": "appendvalue", "applyOn": [{"columnName": "SpecialityCategory", "type": "String", "min": "Ophtamology", "max": "Ophtamology", "mean": "-"}], "functionsData": [{"columnName": "SpecialityCategory", "appendValue": "A", "newColumnName": "Sep", "Column": {"columnName": "SpecialityCategory", "type": "String", "min": "Ophtamology", "max": "Ophtamology", "mean": "-"}}]}], "functionChanges": [{"columnName": "Gender", "functionName": "Append Value", "Type": "String", "Parameters": [{"columnName": "Gender", "appendValue": "S", "newColumnName": "GG", "Column": {"columnName": "Gender", "type": "String", "min": "F", "max": "M", "mean": "-"}}]}, {"columnName": "SpecialityCategory", "functionName": "Append Value", "Type": "String", "Parameters": [{"columnName": "SpecialityCategory", "appendValue": "A", "newColumnName": "Sep", "Column": {"columnName": "SpecialityCategory", "type": "String", "min": "Ophtamology", "max": "Ophtamology", "mean": "-"}}]}], "fileheader": [{"field": "ChargeDetailID", "alias": "ChargeDetailID", "generated": 0, "position": 1, "type": "String"}, {"field": "Age", "alias": "Age", "generated": 0, "position": 2, "type": "real"}, {"field": "Gender", "alias": "Gender", "generated": 0, "position": 3, "type": "String"}, {"field": "DateOfEntry", "alias": "DateOfEntry", "generated": 0, "position": 4, "type": "String"}, {"field": "DateOfService", "alias": "DateOfService", "generated": 0, "position": 5, "type": "String"}, {"field": "ProviderID", "alias": "ProviderID", "generated": 0, "position": 6, "type": "String"}, {"field": "CPTCode", "alias": "CPTCode", "generated": 0, "position": 7, "type": "String"}, {"field": "pdiagnosis1", "alias": "pdiagnosis1", "generated": 0, "position": 8, "type": "String"}, {"field": "pdiagnosis2", "alias": "pdiagnosis2", "generated": 0, "position": 9, "type": "String"}, {"field": "DepartmentCategory", "alias": "DepartmentCategory", "generated": 0, "position": 10, "type": "String"}, {"field": "SpecialityCategory", "alias": "SpecialityCategory", "generated": 0, "position": 11, "type": "String"}, {"field": "days_to_first_post", "alias": "days_to_first_post", "generated": 0, "position": 12, "type": "real"}, {"field": "PayerId", "alias": "PayerId", "generated": 0, "position": 13, "type": "String"}, {"field": "GG", "alias": "GG", "generated": 0, "position": 14, "type": "String"}, {"field": "Sep", "alias": "Sep", "generated": 0, "position": 15, "type": "String"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run 2ndapriltestHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(datapreparation,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "00C8A2A4-6870-424B-94C2-F77590F9D662", "max": "FF814068-2820-4DC9-91BD-DB58F474BF15", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "68.13", "stddev": "16.31", "min": "5.0", "max": "101.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"feature_label": "DateOfEntry", "transformation_label": "Extract Date"}], "feature": "DateOfEntry", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "DateOfEntry"}, {"transformationsData": [{"feature_label": "DateOfService", "transformation_label": "Extract Date"}], "feature": "DateOfService", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "DateOfService"}, {"transformationsData": [{"feature_label": "ProviderID", "transformation_label": "String Indexer"}], "feature": "ProviderID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "03151B5D-685B-4211-B01F-7BBB1C4A8A49", "max": "CBBD4925-1757-45BC-A853-1C2CA203B9E3", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ProviderID"}, {"transformationsData": [{"feature_label": "CPTCode", "transformation_label": "String Indexer"}], "feature": "CPTCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "90932.19", "stddev": "6557.08", "min": "67028", "max": "J9035", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCode"}, {"transformationsData": [{"feature_label": "pdiagnosis1", "transformation_label": "String Indexer"}], "feature": "pdiagnosis1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "B58.01", "max": "T66.XXXD", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis1"}, {"transformationsData": [{"feature_label": "pdiagnosis2", "transformation_label": "String Indexer"}], "feature": "pdiagnosis2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "B20", "max": "Z96.1", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis2"}, {"transformationsData": [{"feature_label": "DepartmentCategory", "transformation_label": "String Indexer"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Cardiology", "max": "Pulmonary", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"feature_label": "SpecialityCategory", "transformation_label": "String Indexer"}], "feature": "SpecialityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Ophtamology", "max": "Ophtamology", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SpecialityCategory"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "days_to_first_post", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "10.28", "stddev": "83.15", "min": "-663.0", "max": "257.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "days_to_first_post"}, {"transformationsData": [{"feature_label": "PayerId", "transformation_label": "String Indexer"}], "feature": "PayerId", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "03E92275-9F7C-4D34-8701-CE4C72CDE9F7", "max": "F71F64B6-E8D9-4A9C-A7E3-A518F957DE52", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PayerId"}, {"transformationsData": [{"feature_label": "GG", "transformation_label": "String Indexer"}], "feature": "GG", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "FS", "max": "MS", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "GG"}, {"transformationsData": [{"feature_label": "Sep", "transformation_label": "String Indexer"}], "feature": "Sep", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "OphtamologyA", "max": "OphtamologyA", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Sep"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run 2ndapriltestHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=linear_regression(autofe, ["ChargeDetailID_stringindexer", "Age", "Gender_stringindexer", "ProviderID_stringindexer", "CPTCode_stringindexer", "pdiagnosis1_stringindexer", "pdiagnosis2_stringindexer", "DepartmentCategory_stringindexer", "SpecialityCategory_stringindexer", "PayerId_stringindexer", "GG_stringindexer", "Sep_stringindexer", "DateOfEntry_dayofmonth", "DateOfEntry_month", "DateOfEntry_year", "DateOfService_dayofmonth", "DateOfService_month", "DateOfService_year"], "days_to_first_post")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)
